# 🚀 llcuda Quickstart Tutorial - Gemma 3-1B Inference on Tesla T4

**Welcome to llcuda!** This notebook shows you how to run fast LLM inference on Google Colab's free Tesla T4 GPU.

## What You'll Learn

✅ Install llcuda (CUDA 12-first inference backend)
✅ Verify Tesla T4 GPU and CUDA 12 compatibility
✅ Load Unsloth's Gemma 3-1B GGUF model
✅ Run fast inference (~45 tokens/second)
✅ Use chat mode for conversations
✅ Benchmark performance

## Requirements

- **GPU**: Tesla T4 (Google Colab free tier)
- **Runtime**: Python 3.10+
- **CUDA**: 12.x (pre-installed on Colab)

---

**Let's get started!** 🎯

## Step 1: Verify GPU and Environment

First, let's make sure you have a Tesla T4 GPU and CUDA 12.

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name,compute_cap,driver_version,memory.total --format=csv

import subprocess
result = subprocess.run(
    ['nvidia-smi', '--query-gpu=name,compute_cap', '--format=csv,noheader'],
    capture_output=True, text=True
)

gpu_info = result.stdout.strip().split(',')
gpu_name = gpu_info[0].strip()
compute_cap = gpu_info[1].strip()

print(f"\n{'='*60}")
print(f"GPU: {gpu_name}")
print(f"Compute Capability: SM {compute_cap}")
print(f"{'='*60}")

if 'T4' in gpu_name and compute_cap == '7.5':
    print("\n✅ Perfect! Tesla T4 (SM 7.5) detected")
    print("   llcuda is optimized for your GPU")
elif compute_cap == '7.5':
    print(f"\n⚠️  {gpu_name} (SM 7.5) - Should work")
else:
    print(f"\n❌ WARNING: SM {compute_cap} detected")
    print("   This tutorial is optimized for SM 7.5 (Tesla T4)")
    print("   Performance may vary on your GPU")

In [ ]:
# Check CUDA version
!nvcc --version | grep "release"

import sys
print(f"\nPython version: {sys.version.split()[0]}")
print(f"Expected: 3.10+ (Colab default)")

## Step 2: Install llcuda

Installing llcuda is simple - just one pip command!

**What happens:**
1. Installs lightweight Python package (~70 KB)
2. On first import, downloads CUDA 12 binaries (~267 MB, one-time)
3. Auto-configures environment for Tesla T4

In [ ]:
!pip install -q llcuda

print("✅ llcuda installed successfully!")

## Step 3: Import llcuda and Download Binaries

First import triggers automatic download of T4-optimized CUDA binaries.

In [ ]:
import llcuda

print(f"\n✅ llcuda version: {llcuda.__version__}")
print("\nBinaries downloaded and configured!")

## Step 4: Verify CUDA Availability

Let's check that llcuda can see your Tesla T4 GPU.

In [ ]:
# Check CUDA availability
cuda_info = llcuda.detect_cuda()

print(f"CUDA available: {cuda_info['available']}")

if cuda_info['available']:
    gpu = cuda_info['gpus'][0]
    print(f"GPU: {gpu['name']}")
    print(f"Compute Capability: SM {gpu['compute_capability']}")
    print(f"Memory: {gpu['memory_total_mb']} MB")
    print("\n✅ llcuda is ready for inference!")
else:
    print("\n❌ CUDA not detected. Please check your GPU runtime.")

## Step 5: Load Gemma 3-1B GGUF Model

We'll use the **Unsloth Gemma 3-1B Instruct** model in GGUF format, quantized with Q4_K_M for optimal balance between speed and quality.

**Model Details:**
- **Name**: unsloth/gemma-3-1b-it-GGUF
- **Quantization**: Q4_K_M (4-bit with K-means)
- **Size**: ~700 MB
- **VRAM**: ~1.2 GB
- **Speed**: ~45 tokens/second on Tesla T4

In [ ]:
# Create inference engine
engine = llcuda.InferenceEngine()

print("✅ Inference engine created!")

In [ ]:
# Load Gemma 3-1B GGUF model from Unsloth
print("📥 Loading Gemma 3-1B Instruct (Q4_K_M)...")
print("   This will download ~700 MB on first run\n")

engine.load_model(
    "unsloth/gemma-3-1b-it-GGUF:gemma-3-1b-it-Q4_K_M.gguf",
    n_gpu_layers=99,  # Offload all layers to GPU
    ctx_size=2048,    # Context window
    n_threads=4       # CPU threads for host operations
)

print("\n✅ Model loaded successfully!")
print("   Ready for inference on Tesla T4")

## Step 6: Your First Inference

Let's run a simple test to see llcuda in action!

In [ ]:
# Simple inference test
prompt = "What is artificial intelligence?"

print(f"Prompt: {prompt}\n")
print("Generating response...\n")

result = engine.infer(
    prompt,
    max_tokens=150,
    temperature=0.7,
    top_p=0.9
)

print("="*60)
print("RESPONSE:")
print("="*60)
print(result.text)
print("="*60)

# Performance metrics
print(f"\n📊 Performance:")
print(f"   Tokens generated: {result.tokens_generated}")
print(f"   Speed: {result.tokens_per_sec:.1f} tokens/second")
print(f"   Latency: {result.latency_ms:.0f} ms")

## Step 7: Interactive Chat Mode

Let's use llcuda's chat engine for multi-turn conversations!

In [ ]:
# Create chat engine
chat = llcuda.ChatEngine(engine)

print("✅ Chat engine ready!")
print("   Multi-turn conversation enabled\n")

In [ ]:
# Example conversation
messages = [
    "Hello! Can you explain what large language models are?",
    "How are they trained?",
    "What are some practical applications?"
]

for i, user_msg in enumerate(messages, 1):
    print(f"\n{'='*60}")
    print(f"Turn {i}")
    print(f"{'='*60}")
    print(f"👤 User: {user_msg}")
    
    response = chat.send(user_msg, max_tokens=100)
    
    print(f"🤖 Gemma: {response.text}")
    print(f"⚡ Speed: {response.tokens_per_sec:.1f} tok/s")

## Step 8: Benchmark Performance

Let's measure inference speed across different prompt lengths.

In [ ]:
import time

# Test prompts of varying lengths
test_cases = [
    ("Short", "What is AI?", 50),
    ("Medium", "Explain the concept of neural networks and how they work.", 100),
    ("Long", "Write a detailed explanation of transformer architecture in machine learning.", 200)
]

print("\n" + "="*70)
print("PERFORMANCE BENCHMARK - Gemma 3-1B Q4_K_M on Tesla T4")
print("="*70)

results = []

for name, prompt, max_tok in test_cases:
    print(f"\n📊 Test: {name} prompt ({len(prompt)} chars, {max_tok} max tokens)")
    
    result = engine.infer(prompt, max_tokens=max_tok, temperature=0.7)
    
    results.append({
        'name': name,
        'tokens': result.tokens_generated,
        'speed': result.tokens_per_sec,
        'latency': result.latency_ms
    })
    
    print(f"   Tokens: {result.tokens_generated}")
    print(f"   Speed: {result.tokens_per_sec:.1f} tok/s")
    print(f"   Latency: {result.latency_ms:.0f} ms")

# Summary
avg_speed = sum(r['speed'] for r in results) / len(results)

print("\n" + "="*70)
print(f"Average Speed: {avg_speed:.1f} tokens/second")
print(f"Expected: ~45 tok/s for Gemma 3-1B Q4_K_M on Tesla T4")
print("="*70)

## Step 9: Try Your Own Prompts!

Now it's your turn! Modify the prompt below and run inference.

In [ ]:
# ✏️ Edit this prompt to try your own!
your_prompt = "Write a haiku about machine learning."

print(f"Your prompt: {your_prompt}\n")

result = engine.infer(
    your_prompt,
    max_tokens=100,
    temperature=0.8,  # Higher = more creative
    top_p=0.9
)

print("="*60)
print(result.text)
print("="*60)
print(f"\n⚡ Generated at {result.tokens_per_sec:.1f} tokens/second")

## Step 10: Advanced Features

### Streaming Inference

Get tokens as they're generated (like ChatGPT).

In [ ]:
from llcuda.jupyter import stream_response

prompt = "Tell me a short story about a robot learning to paint."

print(f"Prompt: {prompt}\n")
print("Response (streaming):\n")

# Stream the response
for chunk in stream_response(engine, prompt, max_tokens=200):
    print(chunk, end='', flush=True)

print("\n\n✅ Streaming complete!")

### Model Information

Get details about the loaded model.

In [ ]:
# Get model information
info = engine.get_model_info()

print("📋 Model Information:")
print("="*60)
print(f"Name: {info.get('name', 'N/A')}")
print(f"Architecture: {info.get('architecture', 'N/A')}")
print(f"Quantization: {info.get('quantization', 'N/A')}")
print(f"Context Size: {info.get('context_size', 'N/A')}")
print(f"Vocab Size: {info.get('vocab_size', 'N/A')}")
print("="*60)

### Engine Statistics

View performance metrics collected during this session.

In [ ]:
# Get engine statistics
stats = engine.get_stats()

print("📊 Session Statistics:")
print("="*60)
print(f"Total requests: {stats.get('total_requests', 0)}")
print(f"Total tokens generated: {stats.get('total_tokens', 0)}")
print(f"Average speed: {stats.get('avg_tokens_per_sec', 0):.1f} tok/s")
print(f"Average latency: {stats.get('avg_latency_ms', 0):.0f} ms")
print(f"Median latency (p50): {stats.get('p50_latency_ms', 0):.0f} ms")
print(f"95th percentile (p95): {stats.get('p95_latency_ms', 0):.0f} ms")
print("="*60)

## Cleanup (Optional)

Stop the inference server to free up GPU memory.

In [ ]:
# Stop the server
engine.stop()

print("✅ Inference server stopped")
print("   GPU memory freed")

## 🎯 Next Steps

Congratulations! You've completed the llcuda quickstart tutorial. Here's what you can try next:

### 1. Try Different Models

llcuda supports any GGUF model from Unsloth or llama.cpp:

```python
# Llama 3.2-3B
engine.load_model("unsloth/Llama-3.2-3B-Instruct-GGUF:Llama-3.2-3B-Instruct-Q4_K_M.gguf")

# Qwen 2.5-7B
engine.load_model("unsloth/Qwen2.5-7B-Instruct-GGUF:qwen2.5-7b-instruct-q4_k_m.gguf")

# Your own fine-tuned model
engine.load_model("/path/to/your/model.gguf")
```

### 2. Explore Quantization Types

Trade speed vs quality:

- **Q4_K_M**: Best balance (default)
- **Q5_K_M**: Higher quality, slower
- **Q8_0**: Near-original quality, slower
- **Q2_K**: Fastest, lower quality

### 3. Integrate with Unsloth Fine-tuning

```python
# After fine-tuning with Unsloth:
# 1. Export to GGUF
model.save_pretrained_gguf("my_model", quantization_method="q4_k_m")

# 2. Load with llcuda
engine.load_model("my_model/my_model-Q4_K_M.gguf")
```

### 4. Production Deployment

Use llcuda's HTTP server API:

```python
from llcuda import ServerManager

server = ServerManager()
server.start(
    model_path="model.gguf",
    n_gpu_layers=99,
    port=8090
)

# Access via HTTP at localhost:8090
```

### 5. Native Tensor Operations (Advanced)

```python
from llcuda.core import Tensor, DType, matmul

# Create tensors on GPU
A = Tensor.zeros([2048, 2048], dtype=DType.Float16, device=0)
B = Tensor.zeros([2048, 2048], dtype=DType.Float16, device=0)

# Matrix multiplication with Tensor Cores
C = A @ B
```

## 📚 Resources

- **GitHub**: https://github.com/waqasm86/llcuda
- **PyPI**: https://pypi.org/project/llcuda/
- **Documentation**: https://github.com/waqasm86/llcuda#readme
- **Unsloth**: https://github.com/unslothai/unsloth
- **llama.cpp**: https://github.com/ggerganov/llama.cpp

## 💬 Support

- **Issues**: https://github.com/waqasm86/llcuda/issues
- **Discussions**: https://github.com/waqasm86/llcuda/discussions

---

**Happy inferencing!** 🚀

*Built with llcuda v2.0.1 - CUDA 12-first inference backend for Unsloth on Tesla T4*